<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width:20%; vertical-align:middle;">
      <img src="LogoUVG.png" width="400"/>
    </td>
    <td style="text-align:left; vertical-align:middle;">
      <h2 style="margin-bottom: 0;">Universidad del Valle de Guatemala - UVG</h2>
      <h3 style="margin-top: 0;">Facultad de Ingeniería - Computación</h3>
      <p style="font-size: 16px; margin-bottom: 0; margin-top: -20px">
        <strong>Curso:</strong> CC3104 - Aprendizaje por Refuerzo 
        <strong>Sección:</strong> 10
      </p>
      <p style="font-size: 16px; margin: 0;"><strong>Agente de trading que aprenda a tomar decisiones de compra, venta o mantenimiento de un activo financiero dentro de un mercado simulado</strong></p>
      <br>
      <p style="font-size: 15px; margin: 0;"><strong>Autores:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px; font-size: 15px;">
        <li>Diego Alexander Hernández Silvestre - <strong>21270</strong></li>
        <li>Linda Inés Jiménez Vides - <strong>21169</strong></li>
        <li>Mario Antonio Guerra Morales - <strong>21008</strong></li>
      </ul>
    </td>
  </tr>
</table>

### Instalación de dependencias e importación de librerías

In [ ]:
#!pip install numpy pandas matplotlib jupyterlab
#!pip install --index-url https://download.pytorch.org/whl/cpu torch
#!pip install --index-url https://download.pytorch.org/whl/cu121 torch # cuda

In [ ]:
# Importar librerías y módulos
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# PyTorch para Deep Q-Network
import torch
import torch.nn as nn
import torch.optim as optim
from dataclasses import dataclass
from typing import Deque, Tuple, List

# Reproducibilidad
SEED = 2000
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Ajustes de gráficos
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.grid"] = True

In [4]:
# Hiperparámetros del entorno y de Deep Q-Network

# Entorno
N_STEPS = 4000 # longitud de pasos en entrenamiento
TEST_STEPS = 1200 # longitud en test
WINDOW = 30 # tamaño de la ventana de observación
TRANS_COST = 0.001 # costo de transacción por cambiar de posición
INIT_CASH = 1_000.0 # efectivo inicial
UNIT_POSITION = 1 # tamaño unitario de posición
ALLOW_SHORT = True # permitir posición -1 (venta en corto)
VOL_SCALE = 0.20 # volatilidad anualizada para Gradient Boosting Models
DRIFT = 0.05 # drift anualizado para GBM

# DQN
GAMMA = 0.99
LR = 1e-3
BATCH_SIZE = 128
REPLAY_SIZE = 50_000
MIN_REPLAY = 2_000 # pasos mínimos antes de entrenar
TARGET_SYNC = 1000 # frecuencia de sync a la target net
EPS_START = 1.0
EPS_END = 0.05
EPS_DECAY_STEPS = 30_000
TRAIN_STEPS = 50_000 # total de pasos de entrenamiento
DOUBLE_DQN = True # usar Double DQN

# Evaluación
RISK_FREE = 0.0 # se asume tasa libre de riesgo 0 para Sharpe simple

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

### Generación de precios para el entorno

In [5]:
# Generación de precios y demás utilidades

def geometric_brownian_motion(
        # Genera una trayectoria GBM discreta:
        # S_{t+1} = S_t * exp((mu - 0.5 sigma^2) dt + sigma * sqrt(dt) * Z_t)
        # # Retorna arreglo de precios (longitud n_steps).
        n_steps: int,
        s0: float = 100.0,
        mu: float = DRIFT,
        sigma: float = VOL_SCALE,
        dt: float = 1/252) -> np.ndarray:
    
    prices = np.zeros(n_steps, dtype=np.float64)
    prices[0] = s0
    for t in range(1, n_steps):
        z = np.random.randn()
        prices[t] = prices[t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * math.sqrt(dt) * z)
    return prices

def to_log_returns(prices: np.ndarray) -> np.ndarray: # Retornos logarítmicos r_t = log(S_t / S_{t-1})
    r = np.zeros_like(prices)
    r[1:] = np.diff(np.log(prices))
    return r

def sharpe_ratio(returns: np.ndarray, risk_free: float = RISK_FREE, eps: float = 1e-9) -> float:
    # Sharpe diario sobre retornos diarios (media - rf) / std.
    if returns.std() < eps:
        return 0.0
    return float((returns.mean() - risk_free) / (returns.std() + eps)) * math.sqrt(252)

def max_drawdown(equity: np.ndarray) -> float: # Máxima caída (drawdown) en porcentaje positivo
    peaks = np.maximum.accumulate(equity)
    drawdowns = 1.0 - (equity / np.maximum(peaks, 1e-9))
    return float(drawdowns.max())

def plot_equity(equity_dict: dict, title="Equity curves"): # Gráfico para curvas de equidad
    for label, eq in equity_dict.items():
        plt.plot(eq, label=label)
    plt.title(title)
    plt.xlabel("Step")
    plt.ylabel("Equity (value)")
    plt.legend()
    plt.show()


### Entorno de trading

In [6]:
# Entorno de trading uniactivo con acciones discretas {sell, hold, buy} -> posiciones {-1, 0, +1}.
# Recompensa = Δ(valor portafolio) neto de costos al cambiar de posición.
# Observación = [ventana de retornos recientes, posición_actual]

class TradingEnv:
    def __init__(self, prices: np.ndarray, window: int = WINDOW, trans_cost: float = TRANS_COST,
                 init_cash: float = INIT_CASH, allow_short: bool = True):
        self.prices = prices.astype(np.float64)
        self.returns = to_log_returns(self.prices)
        self.window = window
        self.trans_cost = trans_cost
        self.init_cash = init_cash
        self.allow_short = allow_short
        self.reset()

    def reset(self):
        self.t = self.window  # primer índice válido para ventana
        self.cash = float(self.init_cash)
        self.position = 0     # -1, 0, +1
        self.equity = [self.cash]
        self.last_price = float(self.prices[self.t-1])
        return self._get_obs()

    def _get_obs(self):
        # obs = [retornos_window, posición_norm]
        window_rets = self.returns[self.t - self.window : self.t]
        pos_norm = np.array([self.position], dtype=np.float32)  # ya está en [-1,0,1]
        obs = np.concatenate([window_rets.astype(np.float32), pos_norm], axis=0)
        return obs

    def step(self, action: int):
        # action in {0,1,2} -> {sell(-1), hold(0), buy(+1)}.
        # Cambiar de posición incurre en costo: trans_cost * precio * |delta_pos| * UNIT_POSITION
        # Recompensa = Δequity (cambio en valor de portafolio) = pnl de la posición + costos (negativos)

        assert action in (0, 1, 2)
        desired_pos = {-1: 0, 0: 1, 1: 2}[self.position]  # mapeo solo para claridad
        target_pos = {-1: 0, 0: 0, 1: 0}  # placeholder

        target = {-1: 0, 0: 0, 1: 0} # ignora

        if action == 0: # sell -> posición -1
            new_pos = -1 if self.allow_short else 0
        elif action == 1: # hold -> mantener
            new_pos = self.position
        else: # buy -> +1
            new_pos = 1

        price_t = float(self.prices[self.t])
        price_tm1 = float(self.prices[self.t - 1])
        ret_t = math.log(price_t / price_tm1 + 1e-12) # retorno log

        # PnL por tener la posición previa en [t-1, t]
        pnl = self.position * ret_t * self.cash  # aproximación: valoriza sobre cash
        # Alternativa más fiel: trackear cantidad de activos; aquí simplificamos para claridad.

        # Costo por cambiar de posición en caso de cambios.
        delta_pos = abs(new_pos - self.position)
        cost = delta_pos * self.trans_cost * price_t * UNIT_POSITION

        # Actualizamos estado financiero
        self.cash += pnl - cost
        self.position = new_pos
        self.equity.append(self.cash)

        # Recompensa = delta equity paso a paso (pnl - cost)
        reward = pnl - cost

        self.t += 1
        done = self.t >= len(self.prices)
        obs = self._get_obs() if not done else None
        info = {"pnl": pnl, "cost": cost, "price": price_t}
        return obs, float(reward), bool(done), info